In [2]:
import re
from time import process_time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# load custom functions and classes
from Utility import Utility
from Sonnet import Sonnet
from Dictionary import Dictionary

In [3]:
import os
for dirname, _, filenames in os.walk('./'):
    if dirname[:6]!='./.git':
        for filename in filenames:
            print(os.path.join(dirname, filename))

./.gitattributes
./CS155_PROJECT3.code-workspace
./Dictionary.py
./HMM.py
./notebook.ipynb
./Sonnet.py
./Untitled.ipynb
./Utility.py
./.ipynb_checkpoints\notebook-checkpoint.ipynb
./.ipynb_checkpoints\Untitled-checkpoint.ipynb
./data\shakespeare.txt
./data\spenser.txt
./data\Syllable_dictionary.txt
./data\syllable_dict_explanation.docx
./__pycache__\Dictionary.cpython-37.pyc
./__pycache__\HMM.cpython-37.pyc
./__pycache__\Sonnet.cpython-37.pyc
./__pycache__\Utility.cpython-37.pyc


In [4]:
shakes = open('./data/shakespeare.txt').read()
print(shakes[:100])

                   1
From fairest creatures we desire increase,
That thereby beauty's rose might nev


There are a few exceptions: Sonnets 99, 126, and 145. Number 99 has fifteen lines. Number 126 consists of six couplets, and two blank lines marked with italic brackets; 145 is in iambic tetrameters, not pentameters. In one other variation on the standard structure, found for example in sonnet 29, the rhyme scheme is changed by repeating the second (B) rhyme of quatrain one as the second (F) rhyme of quatrain three.

Naive parsing from the homework helper ftn

In [6]:
obs, obs_map = Utility.parse_observations(shakes)
print(list(obs_map.keys())[:100])

['1', 'from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'bear', 'memory', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'self', 'foe', 'sweet', 'too', 'cruel', 'art', 'now', 'worlds', 'fresh', 'ornament', 'and', 'only', 'herald', 'gaudy', 'spring', 'within', 'bud', 'buriest', 'content', 'churl', 'makst', 'waste', 'in', 'niggarding', 'pity', 'world', 'or', 'else', 'this', 'glutton', 'be', 'eat', 'due', 'grave', 'thee', '2', 'when', 'forty', 'winters', 'shall', 'besiege', 'brow', 'dig', 'deep', 'trenches', 'field', 'youths', 'proud', 'livery', 'so', 'gazed', 'on', 'will']


Reading the given dictionary

In [8]:
syl_dict = Dictionary.syl_predef()  # load predefined syllable dictionary
syl_dict.head()

,length1,length2
word,,
'gainst,1,0
'greeing,-1,2
'scaped,1,0
'tis,1,0
'twixt,1,0


In [9]:
l = []
for i in range(3205):
    if syl_dict.iloc[i][1]!=0:
        l.append(abs(syl_dict.iloc[i][1])-abs(syl_dict.iloc[i][0]))
print(len(l), max(l), min(l))
"""
Syllable length can vary at most 1
"""

154 1 1


'\nSyllable length can vary at most 1\n'

## Load sonnets from master class 'sonnet'

In [12]:
a = Utility.SonnetLoader('shakespeare')

In [13]:
print(a[1].RhymePair())

[{'now', 'brow'}, {'field', 'held'}, {'lies', 'eyes'}, {'days', 'praise'}, {"excuse'", 'use'}, {'mine', 'thine'}, {'old', 'cold'}]


In [14]:
a[0].SetDict(syl_dict)
a[0].IsRegular()

True

In [15]:
a[98].SetDict(syl_dict)
print(a[98].IsRegular())
print(len(a[98].stringform))            ### Sonnet 99 has 15 lines

False
15


In [16]:
a[125].SetDict(syl_dict)
print(a[125].IsRegular())
print(len(a[125].stringform))           ### Sonnet 125 has 12 lines

False
12


In [17]:
a[0].word_to_index

[[1109, 936, 574, 3025, 692, 1403],
 [2719, 2733, 222, 2262, 1696, 1787, 716],
 [391, 151, 2721, 2247, 2405, 393, 2784, 640],
 [1323, 2707, 1294, 1696, 210, 1323, 1682],
 [391, 2753, 535, 2793, 2741, 1895, 364, 926],
 [983, 2775, 1559, 1024, 3116, 2347, 1116],
 [1642, 5, 948, 3057, 14, 1554],
 [2775, 2339, 2775, 1041, 2793, 2775, 2661, 2339, 2807, 588],
 [2753, 2719, 148, 1816, 2721, 3150, 1104, 1871],
 [113, 1861, 1302, 2793, 2721, 1133, 2533],
 [3119, 2741, 1895, 376, 384, 2775, 531],
 [113, 2707, 460, 1637, 3014, 1398, 1798],
 [1974, 2721, 3149, 1868, 839, 2749, 1168, 208],
 [2793, 827, 2721, 3150, 799, 393, 2721, 1192, 113, 2722]]

In [46]:
from HMM import unsupervised_HMM

safelist = list(range(2,20))
safelist.remove(7)
safelist.remove(13)

a[0].SetDict(syl_dict)
a_all = a[0].word_to_index
for i in safelist:
    a[i].SetDict(syl_dict)
    a_all += a[i].word_to_index

n_states = 4
N_iters = 10

HMM = unsupervised_HMM(a_all, n_states, N_iters)

# Print the transition matrix.
print("Transition Matrix:")
print('#' * 70)
for i in range(len(HMM.A)):
    print(''.join("{:<12.3e}".format(HMM.A[i][j]) for j in range(len(HMM.A[i]))))
print('')
print('')

# Print the observation matrix. 
print("Observation Matrix:  ")
print('#' * 70)
for i in range(len(HMM.O)):
    print(''.join("{:<12.3e}".format(HMM.O[i][j]) for j in range(len(HMM.O[i]))))
print('')
print('')

IndexError: list index out of range